In [ ]:
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb
#import lightgbm as lgb    
#path='C:/Users/mashiji/Desktop'



df=pd.read_csv(r'D:\machine_learning\ccf_first_round_user_shop_behavior.csv')
shop=pd.read_csv(r'D:\machine_learning\ccf_first_round_shop_info.csv')
test=pd.read_csv(r'D:\machine_learning\evaluation_public.csv')
df=pd.merge(df,shop[['shop_id','mall_id']],how='left',on='shop_id')
df['time_stamp']=pd.to_datetime(df['time_stamp'])
train=pd.concat([df,test])
mall_list=list(set(list(shop.mall_id)))
result=pd.DataFrame()
for mall in mall_list:
    train1=train[train.mall_id==mall].reset_index(drop=True) #concat后需要reset_index，不然index会重复，不会整体自然排序      
    l=[]
    wifi_dict = {}
    for index,row in train1.iterrows():
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        for i in wifi_list:
            row[i[0]]=int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]]=1
            else:
                wifi_dict[i[0]]+=1
        l.append(row)    
    delate_wifi=[]
    for i in wifi_dict:
        if wifi_dict[i]<20:
            delate_wifi.append(i)
    m=[]
    for row in l:
        new={}
        for n in row.keys():
            if n not in delate_wifi:
                new[n]=row[n]
        m.append(new)
    train1=pd.DataFrame(m)
    df_train=train1[train1.shop_id.notnull()]
    df_test=train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train['shop_id'].values))
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    
    num_class=df_train['label'].max()+1    
    params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 9,
            'eval_metric': 'merror',
            'seed': 0,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]    
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
    num_rounds=60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    df_test['label']=model.predict(xgbtest)
    df_test['shop_id']=df_test['label'].apply(lambda x:lbl.inverse_transform(int(x)))
    r=df_test[['row_id','shop_id']]
    result=pd.concat([result,r])
    result['row_id']=result['row_id'].astype('int')
    result.to_csv(r'D:\machine_learning\sub.csv',index=False)

C:\Users\georg\Anaconda2\lib\site-packages\ipykernel\__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.199183	test-merror:0.199183
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.160056	test-merror:0.160056
[2]	train-merror:0.139955	test-merror:0.139955
[3]	train-merror:0.126088	test-merror:0.126088
[4]	train-merror:0.114479	test-merror:0.114479
[5]	train-merror:0.107707	test-merror:0.107707
[6]	train-merror:0.098463	test-merror:0.098463
[7]	train-merror:0.092766	test-merror:0.092766
[8]	train-merror:0.086101	test-merror:0.086101
[9]	train-merror:0.081264	test-merror:0.081264
[10]	train-merror:0.076642	test-merror:0.076642
[11]	train-merror:0.069977	test-merror:0.069977
[12]	train-merror:0.065785	test-merror:0.065785
[13]	train-merror:0.061593	test-merror:0.061593
[14]	train-merror:0.057831	test-merror:0.057831
[15]	train-merror:0.054499	test-merror:0.054499
[16]	train-merror:0.052026	test-merror:0.052026
[17]	train-merror:0.049769	test-merror:0.049769

C:\Users\georg\Anaconda2\lib\site-packages\ipykernel\__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\georg\Anaconda2\lib\site-packages\ipykernel\__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.093691	test-merror:0.093691
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.080551	test-merror:0.080551
[2]	train-merror:0.070111	test-merror:0.070111
[3]	train-merror:0.063721	test-merror:0.063721
[4]	train-merror:0.058501	test-merror:0.058501
[5]	train-merror:0.051931	test-merror:0.051931
[6]	train-merror:0.04905	test-merror:0.04905
[7]	train-merror:0.04761	test-merror:0.04761
[8]	train-merror:0.04554	test-merror:0.04554
[9]	train-merror:0.04212	test-merror:0.04212
[10]	train-merror:0.03978	test-merror:0.03978
[11]	train-merror:0.03645	test-merror:0.03645
[12]	train-merror:0.03501	test-merror:0.03501
[13]	train-merror:0.03357	test-merror:0.03357
[14]	train-merror:0.03231	test-merror:0.03231
[15]	train-merror:0.03015	test-merror:0.03015
[16]	train-merror:0.02898	test-merror:0.02898
[17]	train-merror:0.02745	test-merror:0.02745
[18]	train-merror:0.025